# Постановка задачи

## Task #1

**Цель**
Обучаем сеть корректной последовательности действий на бирже: ожидание покупки -> покупка -> ожидание продажи -> продажа.
 
**Observation**
 - Состояние сделки

**Действия**
 1. Ожидание момента для покупки
 2. Покупка (открытие позиции)
 3. Ожидание момента для продажи
 4. Продажа (закрытие позици)

**Награда/штраф**
Действия 1 и 2 возможны при отсутствии открытой позиции. Действия 3 и 4 допустимы только при открытой позиции. При нарушении этого требования - сеть штрафуется.
 
Сеть получает награду при закрытии позиции.

# Импорты

In [1]:
# Системные импорты и настройки
import os
import sys
import yaml
import random
import warnings
import ipynbname
import logging.config

warnings.filterwarnings('ignore')

# for local development
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

# read config
with open('config.yaml', "r") as stream:
    config = yaml.safe_load(stream)
    
# set logging config
log_config = config.get("log", None)
logging.config.dictConfig(log_config)

# set notebook alias
ALIAS = ipynbname.name()
print(ALIAS)

gen12.1-Abstract-02-CloseSignal


In [2]:
# DS frameworks
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

%matplotlib notebook

In [3]:
# NN Frameworks
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D, AveragePooling1D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.models import load_model, clone_model

devices = tf.config.list_physical_devices()
print(devices)

2023-09-23 10:50:47.730229: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
# RT packages
from rl import DQNAgent
from env import TradeEnv
from core_v2 import Constructor, Player


from core_v2.data_point import DataPointFactory

from core_v2.observation_builder.precompute import PrecomputeOrderbookDiffFeature

from train_tools import plot_and_go
from train_tools.train_plot import TrainPlot4
from train_tools.train_manager import TrainManager

In [5]:
seed_value= 0
#os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
#np.random.seed(seed_value)
#tf.random.set_seed(seed_value)

# Конфиг

In [6]:
observation_len = 1

# Параметры точки наблюдения
observation_config = {
    "observation_len": observation_len,             # Количество точек наблюдения в сэмпле
    "offset": observation_len,                      # Количество точек наблюдения в сэмпле
    "future_points": 0,                             # Количество будущех точек для предсказания тренда (временное решение)
    "step_size": 1,                                 # Шаг по датасету
 }

# Датасет

In [7]:
np.random.seed(2)

n_steps = 100
sample_num = 10
sample_size = n_steps//sample_num
safe_interval_size = 3

close_signal = np.empty(0)

sample_size_ = sample_size - safe_interval_size
for i in range(sample_num):
    safe_interval = np.zeros(safe_interval_size)
    total = 0
    while not total==1:
        sample = np.random.uniform(size=sample_size_) > 0.9
        total = sum(sample)

    close_signal = np.concatenate([close_signal, sample, safe_interval])

close_signal = close_signal.reshape(-1,1)
lowest_ask = np.ones(len(close_signal)).reshape(-1,1) * 1
highest_bid = np.ones(len(close_signal)).reshape(-1,1) * 1 + close_signal

dataset = np.concatenate([lowest_ask, highest_bid, close_signal], axis=1)

data_train = pd.DataFrame(dataset, columns=["lowest_ask", "highest_bid", "close_signal"], dtype=np.int8)

#data_train['close_signal'] = data_train['close_signal'].replace(0, -1)
#data_train['close_signal'] = data_train['close_signal'].replace(1, 0.5)
#data_train['close_signal']  = data_train['close_signal']  - 0.5

print(data_train["close_signal"].sum())
print(data_train["close_signal"].count())
data_train.head(14)

10
100


,lowest_ask,highest_bid,close_signal
0,1,1,0
1,1,1,0
2,1,1,0
3,1,1,0
4,1,1,0
5,1,2,1
6,1,1,0
7,1,1,0
8,1,1,0
9,1,1,0


# Инициализация компонентов

## Datapoint factory

In [8]:
dpf_train = DataPointFactory(dataset=data_train, **observation_config)
dpf_test = DataPointFactory(dataset=data_train, **observation_config)

## Env

In [9]:
core_config = {
    "action_controller":{"class": "AbstractTrainController", "params":{ 
            "penalty": -1, 
            "wait_scale": 0, 
            "open_scale": 0, 
            "hold_scale": 0, 
            "close_scale": 1, 
            "last_points_mean": 0
        },},


    "observation_builder":{
        "class": "ObservationBuilder",
        "inputs": [
            {"class": "Input1D", "features": [
                {"class": "RawContextFeature", "params": {"name":"is_open"}},
                {"class": "RawValueFeature", "params": {"name":"close_signal"}}
            ]},
    ]
    }
}
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
core_constructor = Constructor()
env_core = core_constructor.get_core(ALIAS, core_config)

# train environment
env = TradeEnv(env_core, dpf_train, alias=ALIAS, log=True, log_obs=True)

# Нейронная сеть

In [10]:
ACTIVATION = 'relu'
def create_q_model(env):
    num_actions = env.action_space
    #----------------------------------------------
    
    inp_static = Input(shape=env.observation_space[0])
    classif = Dense(32, activation=ACTIVATION)(inp_static)
    classif = Dense(32, activation=ACTIVATION)(classif)
    classif = Dense(32, activation=ACTIVATION)(classif)
    output = Dense(num_actions, activation='softmax')(classif)

    model = Model(inputs=inp_static, outputs=output)
    return model

model = create_q_model(env)
model_target = create_q_model(env)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 32)                96        
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 2,340
Trainable params: 2,340
Non-trainable params: 0
_________________________________________________________________
None


2023-09-23 10:50:55.302766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Обучение

In [14]:
random.seed(seed_value)

core_train = core_constructor.get_core("train", core_config)
env = TradeEnv(core_train, dpf_train, alias=ALIAS, log=True, log_obs=True)

model = create_q_model(env)
model_target = create_q_model(env)
agent = DQNAgent(env, model, model_target)

agent.epsilon_greedy_frames = 2000
agent.epsilon_random_frames = int(0.05 * agent.epsilon_greedy_frames)
agent.max_memory_length = int(1.0 * agent.epsilon_greedy_frames)
agent.max_steps_per_episode = 50000
agent.gamma = 0.95
agent.epsilon_min = 0.01
agent.batch_size = 32
agent.update_after_actions = 4
agent.update_target_network = 1000
agent.loss_function = tf.keras.losses.Huber() #tf.keras.losses.MeanSquaredError()
agent.optimizer = Adam(learning_rate=0.0005, clipnorm=0.001)    #Adam(learning_rate=learning_rate) RMSprop(learning_rate=learning_rate) SGD(learning_rate=learning_rate)


tp = TrainPlot4()
core_test = core_constructor.get_core("test", core_config)
tm = TrainManager(agent, core_test, dpf_test, tp, alias=ALIAS)

In [15]:
tp.init_plot(width=1000, height=800)
tp.update_plot(tm.history)

FigureWidget({
    'data': [{'legendgroup': '1',
              'line': {'color': '#109618', 'width': 1},
              'mode': 'lines',
              'name': 'Train',
              'type': 'scatter',
              'uid': '5bd674be-c730-4665-aff9-4f5768b9a1fb',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'legendgroup': '1',
              'line': {'color': '#FF9900', 'width': 1},
              'mode': 'lines',
              'name': 'Test',
              'type': 'scatter',
              'uid': 'f2bbf3ac-db55-4325-abac-5897c0f1a4ec',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'legendgroup': '2',
              'line': {'color': '#D62728', 'width': 1},
              'mode': 'lines',
              'name': 'Train',
              'type': 'scatter',
              'uid': '44966bf9-7e74-4dca-bef3-59189bf9d870',
              'xaxis': 'x2',
              'yaxis': 'y3'},
             {'legendgroup': '2',
              'line': {'color': '#FF9900'

In [16]:
tm.go(max_frames=7000, test_every=100, snapshot_every=500000, update_plot_every=100, save_since=0.06)

13:24:15 Running reward: -24.80   at episode 11   | frame 1000   | eps: 0.50 | Running loss: 0.15919
13:24:29 Running reward: -16.10   at episode 21   | frame 2000   | eps: 0.01 | Running loss: 0.11702
13:24:43 Running reward: -10.60   at episode 31   | frame 3000   | eps: 0.01 | Running loss: 0.09047
13:24:58 Running reward: -1.67    at episode 41   | frame 4000   | eps: 0.01 | Running loss: 0.04333
13:25:15 Running reward: 3.60     at episode 52   | frame 5000   | eps: 0.01 | Running loss: 0.02396
13:25:34 Running reward: 6.70     at episode 62   | frame 6000   | eps: 0.01 | Running loss: 0.02121
13:25:52 Running reward: 9.17     at episode 72   | frame 7000   | eps: 0.01 | Running loss: 0.02542
done


## Итоги
Конфиг из предыдущей задачи не справился за 10к фреймов. 

- Сделал сеть шире 8х8 -> 32х32: ОК: 8к, 0.9к, 9.4к, 9.6к
- Добавил 3 слой на 32 нейрона : ОК 1.3к, 7.4к, 1.1к, 1.1к

По итогу работает таже схема - сеть погрубже и пошире и алгоритм справляется с задачей более уверенно.
